In [ ]:
# Notbook Eda

In [9]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [10]:
de_desemp = pd.read_csv("./datos/datos_desempenio.csv") 

In [11]:
de_desemp.head(5)

,EmpID,PerfScoreID,PerformanceScore,EngagementSurvey,EmpSatisfaction,SpecialProjectsCount,LastPerformanceReview_Date,DaysLateLast30,Absences
0,10026,4,Exceeds,4.60,5,0,1/17/2019,0,1
1,10084,3,Fully Meets,4.96,3,6,2/24/2016,0,17
2,10196,3,Fully Meets,3.02,3,0,5/15/2012,0,3
3,10088,3,Fully Meets,4.84,5,0,1/3/2019,0,15
4,10069,3,Fully Meets,5.00,4,0,2/1/2016,0,2


In [13]:
de_datos_lab = pd.read_csv("./datos/datos_laborales.csv")
df_lab = de_datos_lab[["EmpID", "DateofHire", "DateofTermination"]]

In [14]:
df_lab.head(5)

,EmpID,DateofHire,DateofTermination
0,10026,7/5/2011,NaN
1,10084,3/30/2015,6/16/2016
2,10196,7/5/2011,9/24/2012
3,10088,1/7/2008,NaN
4,10069,7/11/2011,9/6/2016


In [15]:
de_datos_pers = pd.read_csv("./datos/datos_personales.csv")
df_pers=de_datos_pers[["EmpID", "Sex"]]

In [16]:
df_pers.head(5)

,EmpID,Sex
0,10026,M
1,10084,M
2,10196,F
3,10088,F
4,10069,F


In [17]:
df_tot = pd.merge(df_lab, df_pers, on="EmpID", how="inner")
df_tot.head(3)

,EmpID,DateofHire,DateofTermination,Sex
0,10026,7/5/2011,NaN,M
1,10084,3/30/2015,6/16/2016,M
2,10196,7/5/2011,9/24/2012,F


In [18]:
df_tot["year_hire"] = df_tot["DateofHire"].str.split("/").str[-1]
df_tot["year_term"] = df_tot["DateofTermination"].str.split("/").str[-1]

# Además cambio la columna a tipo entero con el fin de manejar números.
df_tot["year_hire"] = df_tot["year_hire"].astype(int)

# Dado que en year_term tenemos NaN los rellenamos con el año máximo de contratación
# TODO: De hacer ese fillna se puede derivar un error el cual no vamos a tener en
#    consideración para facilitar el ejercicio pero hay que controlar las suposiciones que 
#    se realizan y porque se realizan.
df_tot["year_term"].fillna(df_tot["year_hire"].max(), inplace=True)
df_tot["year_term"] = df_tot["year_term"].astype(int)

df_tot.head(3)

,EmpID,DateofHire,DateofTermination,Sex,year_hire,year_term
0,10026,7/5/2011,NaN,M,2011,2018
1,10084,3/30/2015,6/16/2016,M,2015,2016
2,10196,7/5/2011,9/24/2012,F,2011,2012


In [19]:
# Nos vamos a quedar de nuevo solo con las columnas que nos interesan
df_tot = df_tot[["EmpID", "year_hire", "year_term", "Sex"]]
df_tot.head(3)

,EmpID,year_hire,year_term,Sex
0,10026,2011,2018,M
1,10084,2015,2016,M
2,10196,2011,2012,F


In [21]:
# Sobre este rango de años vamos a calcular la evolución.
# TODO: Aquí hay un posible error que tampoco vamos a considerar, como en 
#    la celda anterior. ¿Sabríais cual es?
hiring_years = df_tot["year_hire"].unique()
hiring_years

array([2011, 2015, 2008, 2012, 2014, 2013, 2009, 2016, 2010, 2018, 2017,
       2007, 2006])

In [22]:
igualdad_info = dict()
for year in hiring_years:
    igualdad_info[year] = df_tot[(df_tot["year_hire"] <= year) & (df_tot["year_term"] > year)] \
                            .groupby(["Sex"])["EmpID"].count().to_dict()
igualdad_info

{2011: {'F': 64, 'M ': 37},
 2015: {'F': 126, 'M ': 103},
 2008: {'F': 3, 'M ': 3},
 2012: {'F': 82, 'M ': 56},
 2014: {'F': 121, 'M ': 95},
 2013: {'F': 98, 'M ': 71},
 2009: {'F': 8, 'M ': 5},
 2016: {'F': 124, 'M ': 97},
 2010: {'F': 13, 'M ': 8},
 2018: {},
 2017: {'F': 122, 'M ': 97},
 2007: {'M ': 3},
 2006: {'M ': 1}}

In [23]:
igualdad = pd.DataFrame(igualdad_info).unstack()  # Mediante esto obtendríamos una Serie
igualdad.index.set_names(["year", "sex"], inplace=True)  # Con esto renombramos el multi-indice
igualdad = igualdad.reset_index()  # Con esto estaríamos convirtiendo a DataFrame
igualdad.rename(columns={0: "total"}, inplace=True)  # Con esto estamos renombrando la columna de total de personas
igualdad.fillna(0, inplace=True)  # Con esto estamos rellenando los posible vacíos

In [24]:
igualdad.head(3)

,year,sex,total
0,2011,F,64.0
1,2011,M,37.0
2,2015,F,126.0


In [25]:
df_grp = igualdad.groupby("year", as_index=False)["total"].sum()
df_grp.head(3)

,year,total
0,2006,1.0
1,2007,3.0
2,2008,6.0


In [27]:
df_ig = pd.merge(igualdad, df_grp, on="year", how="inner", suffixes=("_sex", "_year"))
df_ig.head(3)

,year,sex,total_sex,total_year
0,2011,F,64.0,101.0
1,2011,M,37.0,101.0
2,2015,F,126.0,229.0


In [28]:
df_ig["percent"] = (100 * df_ig["total_sex"] / df_ig["total_year"]).round(2)
df_ig.head(3)

,year,sex,total_sex,total_year,percent
0,2011,F,64.0,101.0,63.37
1,2011,M,37.0,101.0,36.63
2,2015,F,126.0,229.0,55.02
